In [1]:
import pandas as pd, numpy as np, re, json, os, requests
from scrapy.http import HtmlResponse

ModuleNotFoundError: No module named 'scrapy'

In [2]:
home_url="https://www.zomato.com/bangalore/south-bangalore-restaurants"
headers={"User-Agent":
"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36"}

In [36]:
res_id_list=[]
res_name_list=[]
res_area_list=[]
res_type_list=[]
json_data=[]
temp_response=requests.get(home_url, headers=headers)
print(temp_response)
temp_response=HtmlResponse(url=home_url, body=temp_response.text, encoding="utf-8")
for page_i in range(5):
    temp_eles=temp_response.xpath('.//div[@data-res_id]')
    for ele in temp_eles:
        res_id_list.append(ele.xpath('.//@data-res_id').extract()[0])
        res_name_list.append(ele.xpath('.//a[@data-result-type="ResCard_Name"]/text()').extract()[0].strip())
        res_area_list.append(ele.xpath('.//a[contains(@class, "search_result_subzone")]/b/text()').extract()[0])
        res_type_list.append(", ".join(ele.xpath('.//div[contains(@class, "res-snippet-small-establishment")]//a/text()').extract()))
        temp_json={"restaurant_id":res_id_list[-1], "name":res_name_list[-1], 
                   "area":res_area_list[-1], "restaurant_type":res_type_list[-1]}
        json_data.append(temp_json)
    temp_url=home_url+"?page="+str(page_i+2)
    temp_response=requests.get(temp_url, headers=headers)
    print(temp_response)
    temp_response=HtmlResponse(url=temp_url, body=temp_response.text, encoding="utf-8")

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [37]:
print(len(res_id_list))
print(len(res_name_list))
print(len(res_area_list))
print(len(res_type_list))

75
75
75
75


In [38]:
data=pd.DataFrame({"restaurant_id":res_id_list, "name":res_name_list, 
                   "area":res_area_list, "restaurant_type":res_type_list})
print(data.shape)

(75, 4)


In [39]:
data.to_csv("zomato_75res_data.csv", index=False)

In [41]:
json.dump(json_data, open('zomato_75res_data.json', 'w'))